In [1]:
! pip install transformers
! pip install torch

### importing libraries 

In [2]:
import torch
import numpy as np
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
import json
import random
from sklearn.model_selection import train_test_split
from itertools import zip_longest

### defining general variables

In [3]:
#defining global valriables throughout the whole notebook
EPOCH = 10
BATCH_SIZE = 16
MAX_INPUT_LENGTH = 65
MAX_LABEL_LENGTH = 8
MODEL_LINK = "google/flan-t5-small"
FOLD_NUMBER = 0
SEED = 0
srl_data_link = '/kaggle/input/srl-w-cluster-number103k/srl_orl-cluster_number.csv'

In [4]:
def set_seed():
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [5]:
set_seed()
tokenizer = T5Tokenizer.from_pretrained(MODEL_LINK, legacy=False)
model = T5ForConditionalGeneration.from_pretrained(MODEL_LINK).to('cuda')

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

### functions created for reading(get_data) and organize the files (organize_data)

In [6]:
# a funciton to read data off of a database link is here to help getting and organizing data into dataframes
def get_data(address):
    lines = []
    with open(address) as file:
        for line in file:
            x = json.loads(line)
            lines.append(x)
    sentences, orl, sep_sentences = [], [], []
    for i in range(len(lines)):
        sep_sentences.append(lines[i]['sentences'])
        sentences.append(' '.join(lines[i]['sentences']))
        orl.append(lines[i]['orl'])
    dataframe = pd.DataFrame({'sentence': sentences, 'orl': orl, 'sep_sent': sep_sentences})
    return dataframe

# this function is to make a list of the said attribute for later iterations
def list_of(attributes, requested_atr):
    requested_list = []
    for sublist in attributes:
        if sublist[-1] == requested_atr:
            requested_list.append(sublist)
    return requested_list

# this function was made to find target(s)/agent(s) of a dse according to list of attributes
def organize_data(dataframe):
    
    target_column, agent_column, sentence_column, dse_column = [], [], [], []
    
    for i in range(len(dataframe)):
        
        attributes = dataframe['orl'][i]
        sentence   = dataframe['sep_sent'][i]
        target_list= list_of(attributes, 'TARGET')
        agent_list = list_of(attributes, 'AGENT')
        AGENT, DSE, TARGET = '', '', ''
        
        
        for sublist in attributes:
            if sublist[-1] == 'DSE':
                dse_start = int(sublist[0])
                dse_end = int(sublist[1] + 1)
                DSE += ' '.join(sentence[dse_start:dse_end]) + '|'
                
                
                # looking for the targets and agents of this dse that we have found
                for sub_sublist in target_list:
                    if sub_sublist[0] == dse_start and int(sub_sublist[1] + 1) == dse_end:
                        target_start = int(sub_sublist[2])
                        target_end = int(sub_sublist[3] + 1)
                        TARGET += ' '.join(sentence[target_start:target_end]) + ' |'
                if not TARGET:
                    TARGET += ' |'
                
                for sub_sublist in agent_list:
                    if sub_sublist[0] == dse_start and int(sub_sublist[1] + 1) == dse_end:
                        agent_start = int(sub_sublist[2])
                        agent_end = int(sub_sublist[3] + 1)
                        AGENT += ' '.join(sentence[agent_start:agent_end]) + '|'
                if not AGENT:
                    AGENT += ' |'
                # for every iteration of loop over attributes, if a dse is found, then we need to transfer it to new line of a dataframe
                # for each one of the dse(s) i have to add them into a new array so then they can create the correct dataframe
                target_column.append(TARGET)
                agent_column.append(AGENT)
                dse_column.append(DSE)
                sentence_column.append(dataframe['sentence'][i])

    # end of iteration on all sentences
    output_df = pd.DataFrame({'sentence':sentence_column, 'dse':dse_column, 'target':target_column, 'agent':agent_column})
    return output_df

### a function to call for different links of folders with exact process to extract and organize data

In [7]:
def get_files_of_folder(folder_number):
    folder_number = int()
    folder = f"/kaggle/input/ds-json-format/json_format_dataset/{folder_number}"
    dev_df = get_data(f"{folder}/aaai19srl.dev{folder_number}.conll.json")
    df = get_data(f"{folder}/aaai19srl.train{folder_number}.conll.json")
    test_df = get_data(f"{folder}/aaai19srl.test{folder_number}.conll.json")
    return df, dev_df, test_df
df = get_files_of_folder(FOLD_NUMBER)[0]
dev_df = get_files_of_folder(FOLD_NUMBER)[1]
test_df = get_files_of_folder(FOLD_NUMBER)[2]

### getting training/validation/test data into data frames and dividing each of {agent, target, dse} elements

In [8]:
# df = get_data("/kaggle/input/ds-json-format/json_format_dataset/0/aaai19srl.train0.conll.json")

df = organize_data(df)
df

,sentence,dse,target,agent
0,The Kimberley Provincial Hospital said it woul...,would probably know|,whether one of its patients had Congo Fever |,The Kimberley Provincial Hospital|
1,Saeed said indications were that those tests w...,said|,those tests |,Saeed|
2,He said it was his opinion that the patient --...,his opinion|,the patient -- a woman |,He|
3,The woman was admitted to the hospital on Satu...,complaining|,severe joint pains |,The woman|
4,`` Since our technical equipment is far from p...,said|,our technical equipment |we |,Nazarov|
...,...,...,...,...
3530,Benjamin Franklin Federal Savings & Loan Assoc...,plans|,|,it|
3531,thrift said the restructuring should help it m...,said|,the restructuring |,|
3532,Details of the restructuring wo n't be made fi...,approve|,|,regulators|
3533,"Jay Stevens , an analyst with Dean Witter Reyn...",expected|,|,he|


In [9]:
# dev_df = get_data("/kaggle/input/ds-json-format/json_format_dataset/0/aaai19srl.dev0.conll.json")

dev_df = organize_data(dev_df)
dev_df

,sentence,dse,target,agent
0,The owner though that the animal was suffering...,though|,the animal |,The owner|
1,"The owner put down the animal , although the v...",had forbidden|,him to do so |,the vet|
2,"GATUNA , Rwanda , July 6 -LRB- AFP -RRB- - Pre...",soured relations between|,their neighbouring countries |,|
3,"The formerly close allies fell out in 1999 , t...",fell out|,each |,The formerly close allies|
4,"The formerly close allies fell out in 1999 , t...",fell out|mounting rivalry|,each |,The formerly close allies|The formerly close a...
...,...,...,...,...
1504,He said the independent power segment could gr...,said|expect|,the independent power segment |to supply a sig...,He|We|
1505,Westinghouse also expects its international sa...,expects|,|,Westinghouse|
1506,Analysts had expected Consolidated to post a s...,expected|,Consolidated to post a slim profit |,Analysts|
1507,"They have to continue to tighten their belts ,...",said|,They |,"Craig Kloner , an analyst at Goldman , Sachs &..."


In [10]:
# test_df = get_data("/kaggle/input/ds-json-format/json_format_dataset/0/aaai19srl.test0.conll.json")

test_df = organize_data(test_df)
test_df

,sentence,dse,target,agent
0,Private organizations are also being encourage...,are also being encouraged|,help fight sandstorms |,|
1,"It is believed that sand from northern areas ,...",is believed|,"sand from northern areas , especially the Inne...",|
2,Russia favors creation of `` international ins...,favors|,creation of `` international instruments '' to...,Russia|
3,"Moscow , 16 July : The Russian Foreign Ministr...",has called for|,taking into account Russia 's specific interes...,The Russian Foreign Ministry|
4,A Foreign Ministry statement in connection wit...,reads|,taking into account Russia 's interests |,A Foreign Ministry statement|
...,...,...,...,...
888,We will have a reasonably flat year this year ...,says|,reasonably flat year |,Mr. Schwarz|
889,Mr. Schwarz also said costs associated with U....,said|,costs associated with U.S. Trust 's planned mo...,Mr. Schwarz|
890,"Esselte AB , the Stockholm office supplies com...",proposed|,to acquire the 22 % it does n't own of its U.S...,"Esselte AB , the Stockholm office supplies com..."
891,A committee of outside directors for the Garde...,asked|,|,the parent|


In [11]:
def create_dictionary(sentences):
    dictionary = {}
    for sentence in sentences:
        dictionary[sentence] = {
            "dse": {
                "agents": {
                    "actual": [],
                    "predicted": []
                },
                "targets": {
                    "actual": [],
                    "predicted": []
                }
            }
        }
    return dictionary

### Getting evaluation data into df and dividing each of {agent, target, dse} elements

In [12]:
# function which will write prompt for the model according to the sentence and the items in it
def create_prompt(input_df):
    target_prompt, agent_prompt, target, agent, sentence, dse = [], [], [], [], [], []
    for i in range(len(input_df)):
        target_prompt.append(f"sentence is: {input_df['sentence'][i]} this is DSE: {input_df['dse'][i]}. what is the target of sentence?")
        agent_prompt.append(f"sentence is: {input_df['sentence'][i]} this is DSE: {input_df['dse'][i]}. what is the agent of sentence?")
        target.append(input_df['target'][i])
        agent.append(input_df['agent'][i])
        sentence.append(input_df['sentence'][i])
        dse.append(input_df['dse'][i])
    output_df = pd.DataFrame({'sentence':sentence, 'dse':dse, 'target_prompt':target_prompt, 'target':target, 'agent_prompt':agent_prompt, 'agent':agent})
    return output_df

### Change the sentence and dse into a prompt according to information in that row 
#### change will be applied on all three data frames {train, test, validation}

In [13]:
dev_df = create_prompt(dev_df).reset_index(drop=True)
train_df = create_prompt(df).reset_index(drop=True)
test_df = create_prompt(test_df).reset_index(drop=True)

In [14]:
from torch.utils.data import DataLoader, Dataset
# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length4text, max_length4label):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length4text = max_length4text
        self.max_length4label = max_length4label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        agent_prompt = self.data['agent_prompt'][idx]
        target_prompt = self.data['target_prompt'][idx]
        agent = self.data['agent'][idx]
        target = self.data['target'][idx]
        # tokenizing agent prompt
        agent_prompt_encoding = self.tokenizer(agent_prompt, truncation=True, padding='max_length', max_length=self.max_length4text, return_tensors='pt').to("cuda")
        # tokenizing target prompt
        target_prompt_encoding = self.tokenizer(target_prompt, truncation=True, padding='max_length', max_length=self.max_length4text, return_tensors='pt').to("cuda")
        # tokenizing agent
        agent_encoding = self.tokenizer(agent, truncation=True, padding='max_length', max_length=self.max_length4label, return_tensors='pt').to("cuda")
        # tokenizing target
        target_encoding = self.tokenizer(target, truncation=True, padding='max_length', max_length=self.max_length4label, return_tensors='pt').to("cuda")
        return {
            'sentence': self.data['sentence'][idx],
            'dse': self.data['dse'][idx],
            # agent prompt
            'agent_input_id': agent_prompt_encoding['input_ids'].squeeze(),
            'agent_attention_mask': agent_prompt_encoding['attention_mask'].squeeze(),
            # target prompt
            'target_input_id': target_prompt_encoding['input_ids'].squeeze(),
            'target_attention_mask': target_prompt_encoding['attention_mask'].squeeze(),
            #agent
            'agent_id': agent_encoding['input_ids'].squeeze(),
            'agent_mask': agent_encoding['attention_mask'].squeeze(),
            #target
            'target_id': target_encoding['input_ids'].squeeze(),
            'target_mask': target_encoding['attention_mask'].squeeze()
        }

In [15]:
# Define the loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [27]:
def f1_calculator(split_pred_list, split_actual_list):
    matched, percision, recall, f1 = 0, 0, 0, 0
    predicted_len, actual_len      = 0, 0
    
    for actual_sublist, prediction_sublist in zip(split_pred_list, split_actual_list):
        predicted_len += len(prediction_sublist)
        for i in actual_sublist:
            actual_len += 1
            if i in prediction_sublist:
                matched += 1
                
    try:
        precision = matched / predicted_len
        recall = matched / actual_len
        f1 = 2 * (precision * recall) / (precision + recall)
    except ZeroDivisionError:
        f1 = 0

    return f1

In [17]:
def clear_data(text_list):
    list_of_items_in_text = []
    for single_list in text_list:
        for i in single_list.split('|'):
            i = i.strip()  # Assign the stripped version back to `i`
            if i:  # Check if `i` is not empty after stripping
                list_of_items_in_text.append(i)
    return list_of_items_in_text

In [18]:
def evaluate_model(model, dataloader, prompt_type):
    if prompt_type == 'target':
        id_type = 'target_input_id'
        attention_type = 'target_attention_mask'
        output_type = 'target_id'
    elif prompt_type == 'agent':
        id_type = 'agent_input_id'
        attention_type = 'agent_attention_mask'
        output_type = 'agent_id'
    
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        actual_list, prediction_list = [], []
        for batch_idx, batch in enumerate(dataloader):
            
            # Forward pass
            ids = batch[id_type]
            mask = batch[attention_type]
            output_id = batch[output_type]
            
            actuals = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in output_id]
            
            generated_output = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=64, 
              )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_output]
            
            actual_list.append(clear_data(actuals))
            prediction_list.append(clear_data(preds))
            
    return f1_calculator(prediction_list, actual_list)

In [19]:
f1_scores_target, f1_scores_agent = [], []
#           data_loader:train,v_data_loader:validation
def train_data(data_loader, v_data_loader):
    data_list = []
    model.train()
    for epoch in range(EPOCH):
        losses = []
        for batch in data_loader:

            agent_input = batch['agent_input_id']
            agent_attention_mask = batch['agent_attention_mask']

            target_input = batch['target_input_id']
            target_attention_mask = batch['target_attention_mask']

            agent_id = batch['agent_id']
            agent_mask = batch['agent_mask']

            target_id = batch['target_id']
            target_mask = batch['target_mask']

            optimizer.zero_grad()

            agent_output = model(agent_input, attention_mask=agent_attention_mask, labels=agent_id)
            target_output = model(target_input, attention_mask=target_attention_mask, labels=target_id)
            
            # created just for examination
            # has no contribution to accuracy of model output
            temprorarly_pd = pd.DataFrame([{
                'actual_agent': [tokenizer.decode(a) for a in agent_input],
                'agent_model_output': agent_output,
                'actual_target': [tokenizer.decode(c) for c in target_input],
                'target_model_output': target_output
            }])
            # 
            # 
            
            agent_loss = agent_output.loss
            target_loss = target_output.loss
            losses.append(agent_loss.item())
            losses.append(target_loss.item())

            agent_loss.backward()
            target_loss.backward()
            optimizer.step()
            

        f1_4_target = evaluate_model(model, v_data_loader, 'target')
        f1_4_agent = evaluate_model(model, v_data_loader, 'agent')
        
        f1_scores_target.append(f1_4_target)
        f1_scores_agent.append(f1_4_agent)
        
        # report the results of training function.
        print(f'loss: {np.mean(losses)}, f1 for target:{f1_4_target}, f1 for agent:{f1_4_agent} \n end of epoch{epoch}. \n')

# By this box, mpqa/orl data is ready to be combined with different forms of clustering being applied on the rest of the data

### .
### .
### .
### .
## chosing randomly out of SRL_without_anchors for calculating the results
### .
### .
### .
### .

In [20]:
srl = pd.read_csv(srl_data_link)
agents, targets, verbs, sentences, cluster_number = [], [], [], [], []
for i in range(len(srl)):
    
    #single line of data is selected from df
    line = srl.iloc[i]
    
    #check for both ARGS, 
    #if both are empty skips that line.
    #if either one of the args is filled-
    #-process continues.
    if line['ARG0'] or line['ARG1']:
        if line['ARG0']:
            agents.append(line['ARG0'])
        else:
            agents.append("")
        if line['ARG1']:
            targets.append(line['ARG1'])
        else:
            targets.append("")
        sentences.append(line['Sentence'])
        verbs.append(line['Predicate'])
        cluster_number.append(line['Cluster_Number'])

srl_df = pd.DataFrame({'sentences': sentences, 'verbs': verbs, 'agents': agents, 'targets': targets, 'cluster_no':cluster_number}, dtype='object').fillna('')
srl_df

/tmp/ipykernel_30/561452967.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  srl_df = pd.DataFrame({'sentences': sentences, 'verbs': verbs, 'agents': agents, 'targets': targets, 'cluster_no':cluster_number}, dtype='object').fillna('')


,sentences,verbs,agents,targets,cluster_no
0,"Pierre Vinken , 61 years old , will join the b...",join,"Pierre Vinken , 61 years old ,",the board,4
1,"Mr. Vinken is chairman of Elsevier N.V. , the ...",is,,Mr. Vinken,4
2,"Mr. Vinken is chairman of Elsevier N.V. , the ...",publishing,group,,4
3,"Rudolph Agnew , 55 years old and former chairm...",named,,"Rudolph Agnew , 55 years old and former chairm...",2
4,A form of asbestos once used to make Kent ciga...,used,,A form of asbestos,2
...,...,...,...,...,...
103171,`` We emphasize discipline because we know tha...,said,"Andrew Mlangeni , another early Umkhonto leade...",We emphasize discipline because we know that t...,7
103172,`` We want to see Nelson Mandela and all our c...,want,We,to see Nelson Mandela and all our comrades out...,8
103173,`` We want to see Nelson Mandela and all our c...,see,We,Nelson Mandela and all our comrades out of prison,8
103174,`` We want to see Nelson Mandela and all our c...,see,we,them,8


In [21]:
agent_prompts, agents, target_prompts, targets, cluster_no = [], [], [], [], []
for i in range(len(srl_df)):
    line = srl_df.iloc[i]
    agent_prompts.append(f" sentence is: {line['sentences']} find agent for this verb: {line['verbs']}")
    agents.append(line['agents'])
    target_prompts.append(f" sentence is: {line['sentences']} find target for this verb: {line['verbs']}")
    targets.append(line['targets'])
    cluster_no.append(line['cluster_no'])
srl_df = []
srl_df = pd.DataFrame({'agent_prompt': agent_prompts, 'agent': agents, 'target_prompt': target_prompts, 'target': targets, 'cluster_no':cluster_no})

#### randomly selecting number of this dataframe to be fed into the model

In [22]:
def random_sample_df(df, num_rows, SEED=None):
    # Check if num_rows is larger than the available number of rows in the DataFrame
    if num_rows > len(df):
        raise ValueError(f"Requested {num_rows} rows, but the DataFrame only contains {len(df)} rows.")
    
    # Sample the DataFrame and return the result
    sampled_df = df.sample(n=num_rows, random_state=SEED)
    return sampled_df

def random_sample_from_all_clusters(df, num_rows_per_cluster, cluster_column, SEED=None):
    # Check if the cluster_column exists in the DataFrame
    if cluster_column not in df.columns:
        raise ValueError(f"Column '{cluster_column}' not found in DataFrame.")

    # Create an empty list to hold the sampled data for each cluster
    sampled_dfs = []

    # Group the DataFrame by the cluster column
    grouped = df.groupby(cluster_column)

    # Iterate over each cluster and sample rows
    for cluster, group in grouped:
        # Check if the group has enough rows to sample
        if len(group) < num_rows_per_cluster:
            raise ValueError(f"Cluster '{cluster}' has only {len(group)} rows, but {num_rows_per_cluster} were requested.")
        
        # Sample the rows from the current cluster
        sampled_group = group.sample(n=num_rows_per_cluster, random_state=SEED)
        sampled_dfs.append(sampled_group)

    # Concatenate the sampled DataFrames for each cluster
    sampled_df = pd.concat(sampled_dfs).reset_index(drop=True)

    return sampled_df


srl_train_2k = random_sample_df(srl_df, 2000, SEED)
srl_train_2k

,agent_prompt,agent,target_prompt,target,cluster_no
85957,"sentence is: Gold , a closely watched baromet...",,"sentence is: Gold , a closely watched baromet...","Gold , a closely watched barometer of investor...",5
81642,"sentence is: Mr. Krenz , age 52 , is known as...",,"sentence is: Mr. Krenz , age 52 , is known as...",a country,2
84022,sentence is: Its mainline minicomputers and w...,Its mainline minicomputers and word processors,sentence is: Its mainline minicomputers and w...,ground,1
40566,"sentence is: He said Businessland , which ope...",Businessland which,"sentence is: He said Businessland , which ope...",stores,4
56496,sentence is: It did n't release terms of the ...,,sentence is: It did n't release terms of the ...,,0
...,...,...,...,...,...
26866,sentence is: Still to come are big issues by ...,,sentence is: Still to come are big issues by ...,"big issues by Bindal Agro Chem Ltd. , a petroc...",6
33576,sentence is: But he adds : `` Everybody and h...,Everybody and his sister,sentence is: But he adds : `` Everybody and h...,M&A shops,4
79434,sentence is: The volume resulted in dial - to...,,sentence is: The volume resulted in dial - to...,The volume,1
56460,"sentence is: It also charges that Imperial , ...",It,"sentence is: It also charges that Imperial , ...","that Imperial , the holding company for Imperi...",6


In [23]:
def combine_and_shuffle(df1, df2, SEED=None):
    """
    Combines two DataFrames with the same columns and shuffles the rows.

    Args:
        df1 (pd.DataFrame): The first DataFrame.
        df2 (pd.DataFrame): The second DataFrame.
        seed (int, optional): Random seed for reproducibility.

    Returns:
        pd.DataFrame: A new DataFrame with rows from both DataFrames shuffled.
    """
    # Combine the two DataFrames using pd.concat
    combined_df = pd.concat([df1, df2], ignore_index=True)
    
    # Shuffle the combined DataFrame
    shuffled_df = combined_df.sample(frac=1, random_state=SEED).reset_index(drop=True)
    
    return shuffled_df
orl_srl = combine_and_shuffle(train_df, srl_train_2k, SEED)
orl_srl

,sentence,dse,target_prompt,target,agent_prompt,agent,cluster_no
0,Another factor which might deter Beijing 's mi...,readiness to fight|,sentence is: Another factor which might deter ...,Taiwan |,sentence is: Another factor which might deter ...,Washington|,NaN
1,"An understanding that , above anything else , ...",understanding|,"sentence is: An understanding that , above any...","that , above anything else , success requires ...","sentence is: An understanding that , above any...",|,NaN
2,Everything depends on how long it takes for Pe...,said|,sentence is: Everything depends on how long it...,Everything |,sentence is: Everything depends on how long it...,Sader|,NaN
3,NaN,NaN,"sentence is: But Mr. Greenspan , who has repe...",whatever losses,"sentence is: But Mr. Greenspan , who has repe...",,3.0
4,NaN,NaN,sentence is: And the Pentagon said dozens of ...,emergency supplies,sentence is: And the Pentagon said dozens of ...,dozens of additional crews and transport aircraft,7.0
...,...,...,...,...,...,...,...
5530,NaN,NaN,"sentence is: I expected him to say , `` I tol...",I told you so,"sentence is: I expected him to say , `` I tol...",him,6.0
5531,"Naturally , I start crying , and one of my mai...",start crying|,"sentence is: Naturally , I start crying , and ...",|,"sentence is: Naturally , I start crying , and ...",I|,NaN
5532,"`` Watching wildlife can be a lot of fun , '' ...",says|,sentence is: `` Watching wildlife can be a lot...,Watching wildlife |,sentence is: `` Watching wildlife can be a lot...,the sign|,NaN
5533,Rumor has it that the late author of the green...,told|'m a little uneasy|,sentence is: Rumor has it that the late author...,it |when I tell the Americans about my model |,sentence is: Rumor has it that the late author...,the late author of the greenhouse model|I|,NaN


In [24]:
# # Create a data loader for TRAIN dataframe 
# train_dataset = CustomDataset(orl_srl, tokenizer, max_length4text= MAX_INPUT_LENGTH, max_length4label= MAX_LABEL_LENGTH)
# train_data_loader = DataLoader(train_dataset, batch_size= BATCH_SIZE, shuffle=True)

# # Create a data loader for EVALUATION dataframe
# val_dataset = CustomDataset(dev_df, tokenizer, max_length4text= MAX_INPUT_LENGTH, max_length4label= MAX_LABEL_LENGTH)
# val_data_loader = DataLoader(val_dataset, batch_size= BATCH_SIZE, shuffle=False)

# # Create a data loader for TEST dataframe
# test_dataset = CustomDataset(test_df, tokenizer, max_length4text= MAX_INPUT_LENGTH, max_length4label= MAX_LABEL_LENGTH)
# test_data_loader = DataLoader(test_dataset, batch_size= BATCH_SIZE, shuffle=False)


In [25]:
# Create a data loader for TRAIN dataframe 
train_dataset = CustomDataset(train_df, tokenizer, max_length4text= MAX_INPUT_LENGTH, max_length4label= MAX_LABEL_LENGTH)
train_data_loader = DataLoader(train_dataset, batch_size= BATCH_SIZE, shuffle=True)

# Create a data loader for EVALUATION dataframe
val_dataset = CustomDataset(dev_df, tokenizer, max_length4text= MAX_INPUT_LENGTH, max_length4label= MAX_LABEL_LENGTH)
val_data_loader = DataLoader(val_dataset, batch_size= BATCH_SIZE, shuffle=False)

# Create a data loader for TEST dataframe
test_dataset = CustomDataset(test_df, tokenizer, max_length4text= MAX_INPUT_LENGTH, max_length4label= MAX_LABEL_LENGTH)
test_data_loader = DataLoader(test_dataset, batch_size= BATCH_SIZE, shuffle=False)


In [ ]:
train_data(train_data_loader, val_data_loader)

loss: 0.45457534466731064, f1 for target:0.4043790626069107, f1 for agent:0.6194005800837898 
 end of epoch0. 



In [ ]:
import matplotlib.pyplot as plt
# Plot F1 scores for target and agent
epochs = range(0, EPOCH)

plt.figure(figsize=(10, 6))
plt.plot(epochs, f1_scores_target, label='F1 Target', marker='o')
plt.plot(epochs, f1_scores_agent, label='F1 Agent', marker='o')

plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.title('F1 Scores for Target and Agent over Epochs')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
target_accuracy = evaluate_model(model, val_data_loader, 'target')
agent_accuracy = evaluate_model(model, val_data_loader, 'agent')
print(f'f1 Agent: {agent_accuracy}. f1 Target: {target_accuracy}\n\n----------------------')

### mpqa fold 0 to 4 
#### average f1 score for agent and target
fold0_agent = 0.8197088465845465
fold0_target = 0.5834266517357223

fold1_agent = 0.7872340425531915
fold1_target = 0.5599104143337066

fold2_agent = 0.7849944008958566
fold2_target = 0.568868980963046

fold3_agent = 0.7950727883538634
fold3_target = 0.5789473684210527

fold4_agent = 0.7939529675251961
fold4_target = 0.571108622620380